In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.linalg import block_diag

plt.close('all')

Loading the data

In [181]:
data = np.load('data_point_land_1.npz', allow_pickle=True)

#print(data.files)

Meas = data['Meas'] #Landmark measurements
Uf = data['Uf'] #measured forward velocity (odometry)
Ua = data['Ua'] #measured angular velocity (odometry)
Q = data['Q']
Qturn = data['Qturn']
R = data['R']
Nland = data['Nland'] #number of Landmarks
Ts = data['Ts']
Wturn = data['wturn'] #treshold Wturn
Pose = data['Pose'] #data to be used only for comparison (x(t), y(t), theta(t) of the robot)
Landmarks = data['Landmarks'] #data to be used only for comparison (ith row corresponds to i+1th of landmark locations.)

In [182]:
Range_land = Meas.item()['range'] #landmark ranges
Angle_land = Meas.item()['angle']  #landmark angles
Index_land = Meas.item()['land']  #landmark indices

Angle Wrapping Function

In [183]:
def angle_wrap(angle):
    angle = (angle + np.pi) % (2 * np.pi) - np.pi
    return angle

Initialising

In [184]:
N = Uf.shape[0] #Number of odometry measurements

n_upper = 3 #upper system order: x,y,theta
n_lower = Nland.item()*2 #lower system order: 2 for every landmark (x,y)
n = n_upper + n_lower  #system order

x0 = np.zeros(n) #initial states
x0[0] = Pose[0,0] #x(0)
x0[1] = Pose[0,1] #y(0)
x0[2] = Pose[0,2] #theta(0)

#upper covariance
lambda_ = 0.01
P_upper = lambda_ * np.eye(n_upper) #uncertainty of x,y,theta
upper_zeros = np.zeros((n_upper,n_lower))
#lower covariance
eta = 1000
P_lower = eta * np.eye(n_lower) #uncertainty landmarks
lower_zeros = np.zeros((n_lower,n_upper))
#Initial covariance
P0 = np.block([[P_upper,upper_zeros],
              [lower_zeros,P_lower]])

Xp = x0.copy()
Pp = P0.copy()

EKF loop

In [187]:
X_pred = np.empty((N,n))
P_pred = np.empty((N,n))
i = 0
while i < N:
    #prediction
    Xp_lower = np.zeros(n_lower)
    Xp = Xp + np.concatenate((np.array([Uf[i]*np.cos(Xp[2]), Uf[i]*np.cos(Xp[2]), Ua[i]]), Xp_lower))

    dUpper_dx = np.array([[1, 0, -Ts*Uf[i]*np.sin(Xp[2])],
                          [0, 1, Ts*Uf[i]*np.cos(Xp[2])],
                          [0, 0, 1]])
    dLower_dx = np.eye(n_lower)
    F = np.block([[dUpper_dx,upper_zeros],
                [lower_zeros,dLower_dx]])

    dupper_dw = np.array([[-Ts*np.cos(Xp[2]), 0],
                  [-Ts*np.sin(Xp[2]), 0],
                  [0, -Ts]])
    dlower_dw = np.zeros((n_lower, 2))
    G = np.block([[dupper_dw],
                  [dlower_dw]])

    if Ua[i] >= Wturn: #If turn rate is bigger than a given turn rate:
        Q = Qturn

    Pp = F@Pp@F.T + G@Q@G.T


    #correction (I dont know if correction should be before or after prediction. Maybe first a correction during initialization, then prediction then another corretion.)

    i += 1



In [ ]:
#before correction step: dont forget to insert the landmark into state vector the first time. After inserting don't insert the same one again.
if landmark
    L_x[i] = blaba
    L_y[i] = blabla